In [62]:
import pandas as pd
from sqlalchemy import create_engine

data = pd.read_csv('raw1.csv')
data = data.drop(columns=['标题链接']).drop_duplicates(subset='名称').dropna(subset=['ID', '评分'])

# 处理字段1，提取类型年份、状态
def f1(text_):
    parts = text_.replace('，',',').replace(',·', '').split(',')
    if '/' in parts[0] or len(parts)==4:
        return {
            "类型": parts[0].split(' / '),
            "年份": parts[1],
            "状态": parts[2]}
    else: 
        return {
            "类型": '未知',
            "年份": parts[0],
            "状态": parts[1]}
data['字段1'] = data["字段1"].str.replace(',·', '')
d1 = data['字段1'].apply(f1).apply(pd.Series)
d1['状态'] = d1['状态'].str.extract('(已完结|连载中)')[0]
data = pd.concat([data, d1], axis=1)
# 处理年份
data['年份'] = data['年份'].astype(int)
# 处理评分
data['评分'] = data['评分'].apply(lambda a: a.replace('暂无评分', '0').replace('分', '')).astype(float)
# 修正单位
def f2(a):
    a = str(a)
    if '万' in a:
        return int(float(a.replace('万', '')) * 1e4)
    elif '亿' in a:
        return int(float(a.replace('亿', '')) * 1e8)
    else:
        return int(a)
for col in ['点赞', '投币', '收藏', '分享']:
    data[col] = data[col].apply(f2)
# 处理"播放/弹幕/追番"
data['播放'] = data['播放/弹幕/追番'].str.extract(r'((?:\d+(?:\.\d+)?)+[万亿]?)播放')[0].fillna(0).apply(f2)
data['弹幕'] = data['播放/弹幕/追番'].str.extract(r'播放\s*·\s*((?:\d+(?:\.\d+)?)+[万亿]?)弹幕')[0].fillna(0).apply(f2)
data['追番'] = data['播放/弹幕/追番'].str.extract(r'弹幕\s*·\s*((?:\d+(?:\.\d+)?)+[万亿]?)(?:系列)?追番')[0].fillna(0).apply(f2)

# 类型分开存储
category = data[['名称', '类型']].explode('类型')
# 筛选
data = data.drop(columns=['字段1', '播放/弹幕/追番', '类型'])
data = data[(data['评分']>0) & (data['年份']>=2000)]

# 存入mysql数据库
username = 'root'
password = '123456'
host = 'localhost'
my_db = 'my_db'
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{my_db}')
data.to_sql('dongman', engine, index=False, if_exists='replace')
category.to_sql('dongman_cotegory', engine, index=False, if_exists='replace')

7092